<a href="https://colab.research.google.com/github/Debayan2004/BR-Tumor-Segmentation/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import KFold
from scipy.spatial.distance import directed_hausdorff

In [2]:
# Cross-validation function
def cross_validation(X, y, n_splits=5):
    """Perform cross-validation and calculate the metrics for each fold."""
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    dice_scores = []
    volume_similarities = []
    hausdorff_distances = []

    for train_idx, val_idx in kf.split(X):
        # Split data
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        # Build and train the model
        model = build_model(input_shape=X_train.shape[1:])
        model.fit(X_train, y_train, epochs=5, batch_size=16, verbose=1)

        # Make predictions
        y_pred = model.predict(X_val)

        # Calculate metrics
        dice = dice_coefficient(y_val, y_pred)
        volume_sim = volume_similarity(y_val, y_pred)
        hausdorff = hausdorff_distance(y_val, y_pred)

        # Store the metrics for this fold
        dice_scores.append(dice.numpy())
        volume_similarities.append(volume_sim.numpy())
        hausdorff_distances.append(hausdorff)

    # Calculate the mean and std of the metrics
    print("Dice Coefficient: Mean = {:.4f}, Std = {:.4f}".format(np.mean(dice_scores), np.std(dice_scores)))
    print("Volume Similarity: Mean = {:.4f}, Std = {:.4f}".format(np.mean(volume_similarities), np.std(volume_similarities)))
    print("Hausdorff Distance: Mean = {:.4f}, Std = {:.4f}".format(np.mean(hausdorff_distances), np.std(hausdorff_distances)))
